In [ ]:
import zipfile
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

: 

# Análisis de datos

## Importación del Dataset

In [ ]:
# Nombre del archivo zip
zip_path = "archive.zip"

# Extraer todo el contenido
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("diabetes_dataset")

# Verificar archivos extraídos
print("Archivos extraídos:")
print(os.listdir("diabetes_dataset"))


Existen 3 datasets:


* df_original: Muestra más representativa de la realidad por la distribución desbalanceada de clases. La variable objetivo Diabetes_012 tiene 3 clases: 0 indica sin diabetes o diabetes solo durante el embarazo, 1 indica prediabetes y 2 indica diabetes.

* df_balanceado: Muestra balanceada artificialmente al 50% para cada clase. La variable objetivo Diabetes_binary tiene 2 clases: 0 es sin diabetes y 1 es prediabetes o diabetes.

* df_binario:  Muestra no balanceada de encuestas. La variable objetivo Diabetes_binary tiene 2 clases: 0 es sin diabetes y 1 es prediabetes o diabetes.



In [ ]:
# Cargar
df_original = pd.read_csv(r"D:\Principal\ML\diabetes_dataset\diabetes_012_health_indicators_BRFSS2015.csv")

# Vista previa del dataset
df_original.head()

In [ ]:
# Cargar
df_balanceado = pd.read_csv(r"D:\Principal\ML\diabetes_dataset\diabetes_binary_5050split_health_indicators_BRFSS2015.csv")

# Vista previa del dataset
df_balanceado.head()

In [ ]:
# Cargar
df_binario = pd.read_csv(r"D:\Principal\ML\diabetes_dataset\diabetes_binary_health_indicators_BRFSS2015.csv")

# Vista previa del dataset
df_binario.head()

## Análisis de datos

### Caracteristicas principales

In [ ]:
# Obtener distribución en porcentaje por dataset
dist_original = df_original['Diabetes_012'].value_counts(normalize=True).sort_index() * 100
dist_balanceado = df_balanceado['Diabetes_binary'].value_counts(normalize=True).sort_index() * 100
dist_binario = df_binario['Diabetes_binary'].value_counts(normalize=True).sort_index() * 100

# Construcción de la tabla
tabla = pd.DataFrame({
    'df_original (0=sin, 1=prediab, 2=diab)': [
        df_original.shape[0],
        df_original.isnull().sum().sum(),
        dist_original.get(0, 0),
        dist_original.get(1, 0),
        dist_original.get(2, 0)
    ],
    'df_balanceado (0=sin, 1=prediab+diab)': [
        df_balanceado.shape[0],
        df_balanceado.isnull().sum().sum(),
        dist_balanceado.get(0, 0),
        dist_balanceado.get(1, 0),
        0  # Clase 2 no aplica
    ],
    'df_binario (0=sin, 1=prediab+diab)': [
        df_binario.shape[0],
        df_binario.isnull().sum().sum(),
        dist_binario.get(0, 0),
        dist_binario.get(1, 0),
        0  # Clase 2 no aplica
    ]
}, index=[
    "Tamaño del dataset",
    "Total de valores nulos",
    "Clase 0 (%)",
    "Clase 1 (%)",
    "Clase 2 (%)"
])

# Mostrar la tabla redondeada
tabla = tabla.round(2)
from IPython.display import display
display(tabla)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

df_original['Diabetes_012'].value_counts().sort_index().plot(kind='bar', ax=axes[0], title='df_original')
axes[0].set_xlabel("Clase"); axes[0].set_ylabel("Frecuencia"); axes[0].set_ylim(0, 230000)

df_balanceado['Diabetes_binary'].value_counts().sort_index().plot(kind='bar', ax=axes[1], title='df_balanceado')
axes[1].set_xlabel("Clase"); axes[1].set_ylim(0, 230000)

df_binario['Diabetes_binary'].value_counts().sort_index().plot(kind='bar', ax=axes[2], title='df_binario')
axes[2].set_xlabel("Clase"); axes[2].set_ylim(0, 230000)

plt.suptitle("Distribución de la variable objetivo", fontsize=14)

plt.tight_layout()
plt.show()


Distribución de variables



In [ ]:
# Detectar variables binarias (con solo 2 valores únicos)
def get_binarias(df):
    return [col for col in df.columns if df[col].nunique() == 2 and col not in ['Diabetes_012', 'Diabetes_binary']]

# Detectar variables continuas (más de 2 valores únicos)
def get_continuas(df):
    return [col for col in df.columns if df[col].nunique() > 2 and col not in ['Diabetes_012', 'Diabetes_binary']]

# Función para graficar variables binarias en barras apiladas
def plot_barras_binarias(df, nombre):
    vars_binarias = get_binarias(df)
    counts = df[vars_binarias].apply(pd.Series.value_counts).T.fillna(0)
    counts[[0, 1]].plot(kind='bar', stacked=True, figsize=(10, 5), color=['lightgray', 'steelblue'])
    plt.title(f"Distribución de variables binarias - {nombre}")
    plt.xlabel("Variable")
    plt.ylabel("Frecuencia")
    plt.xticks(rotation=45)
    plt.legend(title="Valor")
    plt.tight_layout()
    plt.show()

plot_barras_binarias(df_original, "df_original")
plot_barras_binarias(df_balanceado, "df_balanceado")
#plot_barras_binarias(df_binario, "df_binario")


In [ ]:
def resumen_continuas(df, nombre):
    vars_continuas = get_continuas(df)
    resumen = pd.DataFrame({
        'Variable': vars_continuas,
        'Media': [df[col].mean() for col in vars_continuas],
        'Desviación estándar': [df[col].std() for col in vars_continuas],
        'Mínimo': [df[col].min() for col in vars_continuas],
        'Máximo': [df[col].max() for col in vars_continuas],
    })
    resumen = resumen.round(2)
    print(f"Estadísticas de variables continuas - {nombre}")
    display(resumen)

# Mostrar tabla para cada dataset
resumen_continuas(df_original, "df_original")
resumen_continuas(df_balanceado, "df_balanceado")
#resumen_continuas(df_binario, "df_binario")

In [ ]:
# Función para listar variables no numéricas
def get_categoricas(df, nombre):
    no_numericas = df.select_dtypes(exclude=['number']).columns.tolist()
    print(f"\nVariables no numéricas en {nombre}:")
    if no_numericas:
        for col in no_numericas:
            print(f" - {col}")
    else:
        print(" (No se encontraron variables no numéricas)")

get_categoricas(df_original, "df_original")
get_categoricas(df_balanceado, "df_balanceado")
#get_categoricas(df_binario, "df_binario")


### Correlaciones

In [ ]:
# Obtener variables numéricas comunes (excluyendo variable objetivo)
vars_comunes = [col for col in df_original.columns if col in df_balanceado.columns and col in df_binario.columns]
vars_comunes = [col for col in vars_comunes if col not in ['Diabetes_012', 'Diabetes_binary']]

# Calcular correlaciones individuales
corr_o = df_original[vars_comunes + ['Diabetes_012']].corr()['Diabetes_012'].drop('Diabetes_012')
corr_ba = df_balanceado[vars_comunes + ['Diabetes_binary']].corr()['Diabetes_binary'].drop('Diabetes_binary')
corr_bi = df_binario[vars_comunes + ['Diabetes_binary']].corr()['Diabetes_binary'].drop('Diabetes_binary')

# Unificar el orden por correlación en df_original
corr_o = corr_o.sort_values(ascending=False)
order = corr_o.index.tolist()
corr_ba = corr_ba.loc[order]
corr_bi = corr_bi.loc[order]

# Crear figura con 3 subplots en una fila
fig, axes = plt.subplots(1, 3, figsize=(18, len(order)*0.4), sharey=True)

# Plot para df_original
sns.heatmap(corr_o.to_frame(), annot=True, cmap='coolwarm', center=0, ax=axes[0], cbar=False)
axes[0].set_title("df_original")
axes[0].set_xlabel("Correlación")
axes[0].set_ylabel("Variable")

# Plot para df_balanceado
sns.heatmap(corr_ba.to_frame(), annot=True, cmap='coolwarm', center=0, ax=axes[1], cbar=False)
axes[1].set_title("df_balanceado")
axes[1].set_xlabel("Correlación")
axes[1].set_ylabel("")

# Plot para df_binario
sns.heatmap(corr_bi.to_frame(), annot=True, cmap='coolwarm', center=0, ax=axes[2], cbar_kws={'label': 'Coef. de correlación'})
axes[2].set_title("df_binario")
axes[2].set_xlabel("Correlación")
axes[2].set_ylabel("")

plt.tight_layout()
plt.show()


## Preprocesamiento

Se excluiran de los datasets todas las variables con una correlacion absoluta menor o igual a 0.1.

In [ ]:
# Para df_original
corr_o = df_original.corr()['Diabetes_012'].drop('Diabetes_012')
vars_o = corr_o[abs(corr_o) > 0.1].index.tolist()
df_original_filtrado = df_original[vars_o + ['Diabetes_012']]
print(f"df_original: {len(vars_o)} variables seleccionadas")
print("Variables conservadas:", vars_o)

# Para df_balanceado
corr_ba = df_balanceado.corr()['Diabetes_binary'].drop('Diabetes_binary')
vars_ba = corr_ba[abs(corr_ba) > 0.1].index.tolist()
df_balanceado_filtrado = df_balanceado[vars_ba + ['Diabetes_binary']]
print(f"df_balanceado: {len(vars_ba)} variables seleccionadas")
print("Variables conservadas:", vars_ba)

# Para df_binario
corr_bi = df_binario.corr()['Diabetes_binary'].drop('Diabetes_binary')
vars_bi = corr_bi[abs(corr_bi) > 0.1].index.tolist()
df_binario_filtrado = df_binario[vars_bi + ['Diabetes_binary']]
print(f"df_binario: {len(vars_bi)} variables seleccionadas")
print("Variables conservadas:", vars_bi)


In [ ]:
def preprocesar(df, target_col, usar_estratificacion=True):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    if usar_estratificacion:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

    # Escalado estándar a las variables numéricas
    vars_continuas = get_continuas(df)
    scaler = StandardScaler()
    X_train[vars_continuas] = scaler.fit_transform(X_train[vars_continuas])
    X_test[vars_continuas] = scaler.fit_transform(X_test[vars_continuas])

    return X_train, X_test, y_train, y_test, scaler


# Dataset original (multiclase desbalanceado): SÍ estratificar
Xo_train, Xo_test, yo_train, yo_test, scaler_o = preprocesar(df_original_filtrado, 'Diabetes_012', usar_estratificacion=True)

# Dataset balanceado: NO estratificar
Xb_train, Xb_test, yb_train, yb_test, scaler_b = preprocesar(df_balanceado_filtrado, 'Diabetes_binary', usar_estratificacion=False)

# Dataset binario desbalanceado: SÍ estratificar
Xbi_train, Xbi_test, ybi_train, ybi_test, scaler_bi = preprocesar(df_binario_filtrado, 'Diabetes_binary', usar_estratificacion=True)


# Entrenamiento

In [ ]:
def evaluar_modelos_con_matriz(X_train, X_test, y_train, y_test, nombre_dataset,
                               k, c, kern, depth, crit, min_ss):
    print(f"Evaluación de modelos - {nombre_dataset}")

    modelos = {
        "KNN": KNeighborsClassifier(n_neighbors=k),
        "Regresión Logística": LogisticRegression(C=c, max_iter=100),
        #"SVM": SVC(kernel=kern),
        "Árbol de Decisión": DecisionTreeClassifier(max_depth=depth, criterion=crit, min_samples_split=min_ss)
    }

    fig, axes = plt.subplots(1, 4, figsize=(20, 4))

    for idx, (nombre, modelo) in enumerate(modelos.items()):
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        #acc = accuracy_score(y_test, y_pred)
        if len(y_test.unique()) == 2:
            f1 = f1_score(y_test, y_pred, average='binary')
        else:
          f1 = f1_score(y_test, y_pred, average='macro')

        print(f"\n🔹 Modelo: {nombre}")
        #print(f"Accuracy: {acc:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(classification_report(y_test, y_pred, zero_division=0))

        # Matriz de confusión
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
        axes[idx].set_title(nombre)
        axes[idx].set_xlabel('Predicho')
        axes[idx].set_ylabel('Real')

    plt.suptitle(f"Matrices de confusión - {nombre_dataset}", fontsize=14)
    plt.tight_layout()
    plt.show()

## DF Original

In [ ]:
# Grid search para encontrar los mejores parametros del KNN
params_knn = {'n_neighbors': [2, 3, 5]}
scorer = make_scorer(f1_score, average='macro') # Usar F1 como métrica a optimizar
grid = GridSearchCV(KNeighborsClassifier(), param_grid=params_knn, scoring=scorer, cv=10)
grid.fit(Xo_train, yo_train)

In [ ]:
# Grid search para encontrar los mejores parámetros de regresión logística
params_regresion = {'C': [0.01, 0.1, 1]}
# C : default=1.0
# Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.

grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid=params_regresion, scoring=scorer, cv=10)
grid.fit(Xo_train, yo_train)

In [ ]:
# Grid search para encontrar los mejores parametros del arbol de decision
params_arbol = {
    'max_depth': [7, 10, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid=params_arbol, scoring=scorer, cv=10)
grid.fit(Xo_train, yo_train)

In [ ]:
neighbors = 3
c = 1
kern = 'rbf'
depth = None
crit = 'log_loss'
min_ss = 2

evaluar_modelos_con_matriz(Xo_train, Xo_test, yo_train, yo_test, "df_original",
                                      neighbors, c, kern, depth, crit, min_ss)

## DF Balanceado

In [ ]:
# Mejores parámetros para el dataset balanceado
neighbors = 9
c = 10
kern = 'rbf'
depth = 7
crit = 'entropy'
min_ss = 5

evaluar_modelos_con_matriz(Xb_train, Xb_test, yb_train, yb_test, "df_balanceado",
                                      neighbors, c, kern, depth, crit, min_ss)

## DF binario

In [ ]:
# Mejores parámetros para el dataset binario
neighbors = 3
c = 10
kern = 'rbf'
depth = None
crit = 'entropy'
min_ss = 2

evaluar_modelos_con_matriz(Xbi_train, Xbi_test, ybi_train, ybi_test, "df_binario",
                                      neighbors, c, kern, depth, crit, min_ss)

SVM - Dataset Original (df_original, multiclase desbalanceado)

In [ ]:
# 📦 Importar librerías necesarias
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# ⚠️ Asegúrate de tener ejecutado esto antes:
# Xo_train, Xo_test, yo_train, yo_test, scaler_o = preprocesar(...)

# 🔻 Reducir tamaño del dataset para entrenamiento final (30,000 muestras)
Xo_train_reduced, yo_train_reduced = resample(
    Xo_train, yo_train, n_samples=30000, stratify=yo_train, random_state=42
)

# ✅ Entrenar SVM con los mejores parámetros encontrados
modelo_svm_o = SVC(C=10, kernel='rbf', gamma='auto')
modelo_svm_o.fit(Xo_train_reduced, yo_train_reduced)

# 🧠 Predecir con el conjunto de prueba completo
y_pred_o = modelo_svm_o.predict(Xo_test)

# 📊 Mostrar el reporte de métricas
print("\n📊 Reporte SVM - df_original (entrenado con muestra):")
print(classification_report(yo_test, y_pred_o))

# 🔷 Matriz de confusión
sns.heatmap(confusion_matrix(yo_test, y_pred_o), annot=True, fmt='d', cmap='Blues')
plt.title("SVM - df_original (entrenado con muestra de 30k)")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()


SVM - Dataset Balanceado (df_balanceado, binario balanceado)

In [ ]:
# Grid Search para encontrar los mejores parámetros
param_grid_svm_b = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

grid_svm_b = GridSearchCV(SVC(), param_grid=param_grid_svm_b, cv=5)
grid_svm_b.fit(Xb_train, yb_train)
print("🔍 Mejores parámetros SVM - df_balanceado:", grid_svm_b.best_params_)

# Entrenamiento y evaluación final
modelo_svm_b = SVC(**grid_svm_b.best_params_)
modelo_svm_b.fit(Xb_train, yb_train)
y_pred_b = modelo_svm_b.predict(Xb_test)

print("\n📊 Reporte SVM - df_balanceado:")
print(classification_report(yb_test, y_pred_b))

sns.heatmap(confusion_matrix(yb_test, y_pred_b), annot=True, fmt='d', cmap='Blues')
plt.title("SVM - df_balanceado")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()


SVM - Dataset Binario Real (df_binario, binario desbalanceado)

In [ ]:
# Grid Search para encontrar los mejores parámetros
param_grid_svm_bi = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

grid_svm_bi = GridSearchCV(SVC(), param_grid=param_grid_svm_bi, cv=5)
grid_svm_bi.fit(Xbi_train, ybi_train)
print("🔍 Mejores parámetros SVM - df_binario:", grid_svm_bi.best_params_)

# Entrenamiento y evaluación final
modelo_svm_bi = SVC(**grid_svm_bi.best_params_)
modelo_svm_bi.fit(Xbi_train, ybi_train)
y_pred_bi = modelo_svm_bi.predict(Xbi_test)

print("\n📊 Reporte SVM - df_binario:")
print(classification_report(ybi_test, y_pred_bi))

sns.heatmap(confusion_matrix(ybi_test, y_pred_bi), annot=True, fmt='d', cmap='Blues')
plt.title("SVM - df_binario")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()
